In [1]:
import numpy as np
import pandas as pd
rna_local = pd.read_csv('/home/ztang/multitask_RNA/data/RNA_loc/TableS2.csv',index_col = 0)

In [2]:
rna_local.columns

Index(['gene name', 'position in 3UTR', 'subset', 'changes',
       'logFC(neurite/soma) - CAD', 'p-value logFC(neurite/soma) - CAD',
       'logFC(neurite/soma) - Neuro-2a',
       'p-value logFC(neurite/soma) - Neuro-2a', 'logFC(4h/0h ActD)',
       'p-value logFC(4h/0h ActD)', 'logFC(24h/0h ActD)',
       'p-value logFC(24h/0h ActD)', 'number of reads - CAD',
       'minimum number of reads - CAD', 'number of reads - Neuro-2a',
       ' minimum number of reads - Neuro-2a',
       'full library sequence (primers-barcode-test sequence)'],
      dtype='object')

In [3]:
def onehotcoding(seq):
    IN_MAP = np.asarray([[0, 0, 0, 0],
                     [1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    seq = seq.upper().replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4').replace('N', '0')
    return IN_MAP[np.array(list(seq)).astype(int)]

In [11]:
x = []
y = []
for (i,row) in rna_local.iterrows():
    seq = row['full library sequence (primers-barcode-test sequence)'][30:-18]
    vector = np.array([row['logFC(neurite/soma) - CAD'], row['logFC(neurite/soma) - Neuro-2a']])
    p_value =np.array([row['p-value logFC(neurite/soma) - CAD'], row['p-value logFC(neurite/soma) - Neuro-2a']])
    onehot = onehotcoding(seq)
    x.append(onehot)
    temp = np.array([0,0])
    if (p_value < 0.05).all() and (vector>0).all():
        temp[0] = 1
    elif (p_value < 0.05).all() and (vector<0).all():
        temp[1] = 1
        # print(vector)
        # print(p_value)
    y.append(temp)

In [5]:
import h5py
file_name = '/home/ztang/multitask_RNA/data/RNA_loc/RNAloc.h5'
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('x',data = np.array(x))
h5f.create_dataset('y',data =np.array(y))
h5f.close()